# tworzenie plikow nifti i masek

In [1]:
# instalacja i zaimportowanie potrzebnych pakietów
!pip install pyradiomics
!pip install dicom2nifti
import radiomics
import os
import dicom2nifti
import nibabel as nb
from nilearn import image
from nilearn.image import math_img
import glob

train_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/*/FLAIR'
all_patients = sorted(glob.glob(train_path)) # posortowanie ścieżek do folderów FLAIR dla każdego pacjenta
for i, patient in enumerate(all_patients): 

    # przekształcenie serii DICOM na jeden plik NIFTI i zapisanie pod odpowiednią nazwą z nr pacjenta
    dicom2nifti.dicom_series_to_nifti(patient, os.path.join('./',str(i)+'_patient_FLAIR.nii.gz'))

    masker = nb.load(f'./{str(i)}_patient_FLAIR.nii.gz') # załadowanie obrazu NIFTI mózgu
    mask = math_img("img>0", img=masker) # stworzenie maski - wyodrębnienie mózgu od tła na obrazie (tło ma kolor czarny, a kolor czarny to 0)
    nb.save(mask, f'./{str(i)}_mask_FLAIR.nii.gz') # zapisanie maski mózgu pod odpowiednią nazwą z nr pacjenta 

## stworzone pliki NIFTI zostały zapisane na lokalnym komputerze i wstawione tutaj na kaggle, aby nie musieć wykonywać przetwarzania dicom --> nifti za każdym razem jak zatrzyma nam się sesja na kaggle

## Kaggle pozwala na stworzenie folderu o maksymalnej liczbie plików 1000, a my mamy 585*2 plików, ponieważ wszyscy pacjenci + dla każdego maska mózgu... Więc zostały wstawione tutaj dwa foldery ze stworzonymi wcześniej plikami NIFTI o nazwach: 

### nifti-images-masks-bez499 oraz nifti-images-masks-do584

In [ ]:
# załadowanie obrazu NIFTI mózgu i jego maski pierwszego pacjenta
img = nb.load('../input/nifti-images-masks-bez499/0_patient_FLAIR.nii')
masker = nb.load('../input/nifti-images-masks-bez499/0_mask_FLAIR.nii')
masker

In [ ]:
from nilearn import plotting
import matplotlib
import matplotlib.pyplot as plt

# wyświetlenie przekroju mózgu dla pierwszego pacjenta, aby sprawdzić czy plik NIFTI został prawidłowo stworzony
figure, axes = plt.subplots(1, 1,figsize=(15, 7))
plotting.plot_img(img, axes=axes,annotate=False,  display_mode='mosaic')

In [ ]:
# wyświetlenie przekroju maski mózgu dla pierwszego pacjenta, aby sprawdzić czy plik NIFTI maski został prawidłowo stworzony
figure, axes = plt.subplots(1, 1,figsize=(15, 7))
plotting.plot_img(masker, axes=axes,annotate=False,  display_mode='mosaic')

In [ ]:
# sprawdzenie czy obraz mózgu i obraz jego maski mają takie same kształty - mają
print(img.shape)
print(masker.shape)

# Ekstrakcja cech 

In [ ]:
# instalacja i import biblioteki pyradiomics potrzebnej do ekstrakcji cech z obrazów NIFTI
!pip install pyradiomics
import radiomics

from radiomics import featureextractor  
extractor = featureextractor.RadiomicsFeatureExtractor() # stworzenie instancji ekstraktora
# wartości domyślne pozwalają na ustawienie dostępności wszystkich funkcji do ekstrakcji

In [ ]:
# pokazanie które z klas funkcji są dostępne - wszystkie
extractor.enabledFeatures

In [ ]:
# posortowanie plików przed ekstrakcją
import re
import glob

# najpierw ekstrakcja cech z plików z folderu nifti-images-masks-bez499...
# a potem ekstrakcja cech z plików z folderu nifti-images-masks-do584...

nifti_images= sorted(glob.glob(f'../input/nifti-images-masks-do584/*_patient_FLAIR.nii'))
nifti_images.sort(key=lambda f: int(re.sub('\D', '', f)))

nifti_mask= sorted(glob.glob(f'../input/nifti-images-masks-do584/*_mask_FLAIR.nii'))
nifti_mask.sort(key=lambda f: int(re.sub('\D', '', f)))

In [ ]:
import pandas as pd

# ekstrakcja tylko jednego pliku potrzebna do stworzenia ramki danych której nazwy kolumn będą ustawiane po stworzeniu ramek danych z  wyekstrachowanymi cechami dla kolejnych pacjentów
features_flair2= (extractor.execute('../input/nifti-images-masks-bez499/100_patient_FLAIR.nii','../input/nifti-images-masks-bez499/100_mask_FLAIR.nii'))
df2 = pd.DataFrame([features_flair2]) # stworzenie ramki danych z wyekstrahowanymi cechami 

data_list=[]

for i, j in zip(nifti_images, nifti_mask):
    features_flair= pd.Series(extractor.execute(i,j)) # ekstrakcja cech z obrazów
    df = pd.DataFrame(features_flair) # stworzenie ramki danych z wyekstrahowanymi cechami
    df=df.T # transponowanie ramki danych 
    df.columns = df2.columns # ustawienie nazw kolumn na takie same jak nazwy kolumn z ramki danych stworzonej po ekstrakcji cech przed pętlą
    data_list.append(df) # dołączenie ramki danych z wyekstrahowanymi cechami do listy
final_data_day = pd.concat(data_list) # połączenie ramek danych w jedną
final_data_day.to_csv('wszystkie.csv') # zapisanie ramki do pliku csv


# dla 9 pacjentów ekstrakcja nie działa prawidłowo z powodu błędów 
# niezgodności geometrii między obrazem, a maską, a także w 2 przypadkach z powodu braku znalezionej etykiety w masce

#od 0 : 67 nie działa (geometry tolerance)
#od 0 : 71 nie dziala (nothing is segmented)
# od 0 : 314 nie działa (geometry tolerance)
# od 0 : 400 nie dziala (geometry tolerance)
# od 0 : 406 nie dziala (geometry tolerance)
# od 0 : 488 nie dziala (nothing is segmented)
# od 0 : 533 nie dziala (geometry tolerance)
# od 0 : 552 nie działa (geometry tolerance)
# od 0 : 560 nie działa (geometry trolerance)

# te 9 przypadków pomijam i robię ekstrakcję dla reszty 576 pacjentów

# po ominięciu ktoregoś z tych 9 pacjentów w pętli for ustawiam nifti_images[numer od którego ekstrakcja działa poprawnie :]
# i tak samo dla nifti_masks

# dane pobieram i łącze w excel 